In [1]:
import tensorflow as tf
import datetime
import os
import numpy as np
import ctypes
from PIL import Image

In [2]:
#### ------------------------------------------- ####
#### ------------- LOAD THE DATA --------------- ####
#### ------------------------------------------- ####

In [3]:
MAIN_DATASET_FOLDER = "..\\dataset\\"

TEST_SUBFOLDER = os.path.join(MAIN_DATASET_FOLDER, "test")
TRAIN_SUBFOLDER = os.path.join(MAIN_DATASET_FOLDER, "train")

TEST_ESPAGNE_FOLDER = os.path.join(TEST_SUBFOLDER, "espagne")
TEST_FRANCE_FOLDER = os.path.join(TEST_SUBFOLDER, "france")
TEST_JAPON_FOLDER = os.path.join(TEST_SUBFOLDER, "japon")

TRAIN_ESPAGNE_FOLDER = os.path.join(TRAIN_SUBFOLDER, "espagne")
TRAIN_FRANCE_FOLDER = os.path.join(TRAIN_SUBFOLDER, "france")
TRAIN_JAPON_FOLDER = os.path.join(TRAIN_SUBFOLDER, "japon")


In [4]:
def fill_x_and_y_with_images_and_labels(folder, classe, dataset):

    for file in os.listdir(folder):

        file_path = os.path.join(folder, file)
        image = Image.open(file_path)
        image = image.resize((64, 64))
        if image.mode in ("RGB", "P"): image = image.convert("RGBA")
        im_arr = np.array(image).flatten()
        im_arr = im_arr / 255.0

        completed_dataset = {}
        completed_dataset["value"] = im_arr
        completed_dataset["classe"] = classe

        dataset.append(completed_dataset.copy())

In [5]:
def import_dataset(folder, classe):

    dataset = []

    fill_x_and_y_with_images_and_labels(folder, classe, dataset)

    return dataset

In [6]:
x_test = []
y_test = []

x_train = []
y_train = []

class_espagne_expected_outputs = [1, 0, 0]
dataset_test_for_espagne = import_dataset(TEST_ESPAGNE_FOLDER, class_espagne_expected_outputs)
dataset_train_for_espagne = import_dataset(TRAIN_ESPAGNE_FOLDER, class_espagne_expected_outputs)

class_france_expected_outputs = [0, 1, 0]
dataset_test_for_france = import_dataset(TEST_FRANCE_FOLDER, class_france_expected_outputs)
dataset_train_for_france = import_dataset(TRAIN_FRANCE_FOLDER, class_france_expected_outputs)

class_japon_expected_outputs = [0, 0, 1]
dataset_test_for_japon = import_dataset(TEST_JAPON_FOLDER, class_japon_expected_outputs)
dataset_train_for_japon = import_dataset(TRAIN_JAPON_FOLDER, class_japon_expected_outputs)

final_dataset_test = dataset_test_for_espagne + dataset_test_for_france + dataset_test_for_japon
final_dataset_train = dataset_train_for_espagne + dataset_train_for_france + dataset_train_for_japon

In [7]:
for value in final_dataset_test:

    the_image = value["value"].tolist()
    the_classe = value["classe"]

    x_test.append(the_image)
    y_test.append(the_classe)

for value in final_dataset_train:

    the_image = value["value"].tolist()
    the_classe = value["classe"]

    x_train.append(the_image)
    y_train.append(the_classe)

dataset_inputs = x_train
dataset_outputs = y_train

initiate_dataset_inputs_size = len(dataset_inputs[0])
initiate_dataset_inputs_type = ctypes.c_float * initiate_dataset_inputs_size

classes_initiate_dataset_inputs_size = initiate_dataset_inputs_size * 2

In [8]:
inputs_result = []

for sublist in dataset_inputs:
    for item in sublist:
        inputs_result.append(item)

dataset_inputs = inputs_result

outputs_result = []

for sublist in dataset_outputs:
    for item in sublist:
        outputs_result.append(item)

dataset_outputs = outputs_result

arr = [initiate_dataset_inputs_size, classes_initiate_dataset_inputs_size, 3]
arr_size = len(arr)
arr_type = ctypes.c_int * arr_size

In [9]:
# Convert train/test to array
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [10]:
#### ------------------------------------------- ####
#### ------------LOAD MODELS---------------------#### 
#### ------------------------------------------- ####

In [11]:
model_pmc_a = tf.keras.models.load_model("../models/keras_models/model_a.h5")

In [12]:
model_pmc_b = tf.keras.models.load_model("../models/keras_models/model_b.h5")

In [13]:
model_pmc_c = tf.keras.models.load_model("../models/keras_models/model_c.h5")

In [14]:
model_pmc_d = tf.keras.models.load_model("../models/keras_models/model_d.h5")

In [57]:
#### ------------------------------------------- ####
#### ------ DEFINE THE TRAINING ALGORITHM ------ #### 
#### ------------------------------------------- ####

In [15]:
OPTIMIZER = tf.keras.optimizers.SGD(learning_rate=1e-3)
LOSS = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
METRICS = [
    tf.keras.metrics.CategoricalAccuracy(name='acc'),
]

In [16]:
model_pmc_a.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

model_pmc_b.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

model_pmc_c.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

model_pmc_d.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

In [18]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (A) ------------- #### 
#### ------------------------------------------- ####

In [17]:
tf.keras.backend.clear_session()

In [18]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_a.h5', save_best_only=True)

In [19]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_a_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_a = model_pmc_a.fit(
    x=x_train, 
    y=y_train, 
    epochs=100, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
        
        
    ]

)

Epoch 1/100
19/19 [==============================] - 1s 37ms/step - loss: 0.3072 - acc: 0.8917 - val_loss: 0.5409 - val_acc: 0.7333
Epoch 2/100
19/19 [==============================] - 0s 4ms/step - loss: 0.2763 - acc: 0.9067 - val_loss: 0.5535 - val_acc: 0.7667
Epoch 3/100
19/19 [==============================] - 0s 4ms/step - loss: 0.2951 - acc: 0.8967 - val_loss: 0.7359 - val_acc: 0.7250
Epoch 4/100
19/19 [==============================] - 0s 4ms/step - loss: 0.3120 - acc: 0.8783 - val_loss: 0.7535 - val_acc: 0.7000
Epoch 5/100
19/19 [==============================] - 0s 4ms/step - loss: 0.2771 - acc: 0.9033 - val_loss: 0.5556 - val_acc: 0.7250
Epoch 6/100
19/19 [==============================] - 0s 4ms/step - loss: 0.3056 - acc: 0.8900 - val_loss: 0.5747 - val_acc: 0.7000
Epoch 7/100
19/19 [==============================] - 0s 4ms/step - loss: 0.3045 - acc: 0.8867 - val_loss: 0.5660 - val_acc: 0.7000
Epoch 8/100
19/19 [==============================] - 0s 4ms/step - loss: 0.2939 - 

In [20]:
history_a.history.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [23]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (B) ------------- #### 
#### ------------------------------------------- ####

In [24]:
tf.keras.backend.clear_session()

In [70]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_b.h5', save_best_only=True)

In [71]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_b_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_b = model_pmc_b.fit(
    x=x_train, 
    y=y_train, 
    epochs=100, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
    ]
)

Epoch 1/100
19/19 [==============================] - 1s 59ms/step - loss: 1.0980 - acc: 0.3900 - val_loss: 0.9922 - val_acc: 0.6000
Epoch 2/100
19/19 [==============================] - 0s 4ms/step - loss: 0.9866 - acc: 0.5433 - val_loss: 0.9476 - val_acc: 0.5333
Epoch 3/100
19/19 [==============================] - 0s 4ms/step - loss: 0.9550 - acc: 0.5717 - val_loss: 0.8831 - val_acc: 0.6000
Epoch 4/100
19/19 [==============================] - 0s 4ms/step - loss: 0.8357 - acc: 0.6333 - val_loss: 0.8064 - val_acc: 0.6500
Epoch 5/100
19/19 [==============================] - 0s 4ms/step - loss: 0.7948 - acc: 0.6767 - val_loss: 0.8040 - val_acc: 0.6333
Epoch 6/100
19/19 [==============================] - 0s 4ms/step - loss: 0.8169 - acc: 0.6350 - val_loss: 0.7464 - val_acc: 0.6750
Epoch 7/100
19/19 [==============================] - 0s 4ms/step - loss: 0.7745 - acc: 0.6817 - val_loss: 0.7181 - val_acc: 0.7333
Epoch 8/100
19/19 [==============================] - 0s 4ms/step - loss: 0.7395 - 

In [25]:
history_b.history.keys()

NameError: name 'history_b' is not defined

In [28]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (C) ------------- #### 
#### ------------------------------------------- ####

In [29]:
tf.keras.backend.clear_session()

In [72]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_c.h5', save_best_only=True)

In [73]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_c_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_c = model_pmc_c.fit(
    x=x_train, 
    y=y_train, 
    epochs=100, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
    ]
)

Epoch 1/100
19/19 [==============================] - 1s 59ms/step - loss: 1.0398 - acc: 0.5417 - val_loss: 0.8518 - val_acc: 0.5917
Epoch 2/100
19/19 [==============================] - 0s 5ms/step - loss: 0.9069 - acc: 0.5683 - val_loss: 0.7884 - val_acc: 0.6667
Epoch 3/100
19/19 [==============================] - 0s 5ms/step - loss: 0.8530 - acc: 0.6117 - val_loss: 0.7427 - val_acc: 0.7000
Epoch 4/100
19/19 [==============================] - 0s 4ms/step - loss: 0.7808 - acc: 0.6600 - val_loss: 0.6635 - val_acc: 0.7333
Epoch 5/100
19/19 [==============================] - 0s 5ms/step - loss: 0.7654 - acc: 0.6667 - val_loss: 0.7076 - val_acc: 0.6667
Epoch 6/100
19/19 [==============================] - 0s 5ms/step - loss: 0.7531 - acc: 0.6800 - val_loss: 0.7058 - val_acc: 0.7000
Epoch 7/100
19/19 [==============================] - 0s 4ms/step - loss: 0.6945 - acc: 0.7217 - val_loss: 0.7159 - val_acc: 0.7250
Epoch 8/100
19/19 [==============================] - 0s 5ms/step - loss: 0.6987 - 

In [23]:
history_c.history.keys()

NameError: name 'history_c' is not defined

In [ ]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (D) ------------- #### 
#### ------------------------------------------- ####

In [26]:
tf.keras.backend.clear_session()

In [27]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_d.h5', save_best_only=True)

In [67]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_d_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_d = model_pmc_d.fit(
    x=x_train, 
    y=y_train, 
    epochs=100, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
    ]
)

Epoch 1/100
19/19 [==============================] - 1s 57ms/step - loss: 0.1334 - acc: 0.9650 - val_loss: 0.6816 - val_acc: 0.7333
Epoch 2/100
 1/19 [>.............................] - ETA: 0s - loss: 0.0911 - acc: 0.9688

KeyboardInterrupt: 

In [62]:
history_d.history.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [30]:
#### ------------------------------------------- ####
#### ------------- SHOW CURVES ------------------#### 
#### ------------------------------------------- ####

In [69]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [73]:
tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 11448), started 0:05:49 ago. (Use '!kill 11448' to kill it.)